In [1]:
import requests
BASE_URL = 'http://localhost:5000'

def report(msg, vector):
    requests.get(BASE_URL + '/spam/report',
             json={'vector': vector,
                   'message': msg}
            ).ok

def is_spam(vector):
    data = requests.get(BASE_URL + '/spam/detect',
                        params={'vector': ','.join(str(v) for v in vector)}
            ).json()
    return data['spam'], data['confidence']

def get_word_vectors(indexes):
    data = requests.get(BASE_URL + '/words/vector',
                        params={'ids': ','.join(str(i) for i in indexes)}).json()
    return {int(k): v['vector'] for k,v in data['words'].items()}

def get_word_indexes():
    """return a dict of word to index."""
    content = requests.get(BASE_URL + '/words/list').content
    return {word: i for i, word in enumerate(content.split('\n'))}

In [2]:
WORD_INDEXES = get_word_indexes()


def calc_message(message):
    import random
    import numpy as np
    
    indexes = [WORD_INDEXES[word] 
               for word in message.split(' ')
               if word in WORD_INDEXES]
    fakes = [random.randint(0, len(WORD_INDEXES)) for _ in range(20)]
    
    shuffled = indexes + fakes
    random.shuffle(shuffled)
    vectors = get_word_vectors(shuffled)
    vec_sum = sum(np.array(vectors[i]) for i in indexes).tolist()
    return vec_sum

In [3]:
spams = [
    'Want to see my nudes? meet me at http://www.rounds.com. see you around!',
    'come see me on camera just add your credit card http://getsomenudes.com',
    'free pics and live video come here bit.ly/ERERERERE'
]
good = [
    'Hi I am your mother and I love you',
    'Did you watch the movie last night?',
    'Whats up with all the weird clouds?',
]

In [38]:
def is_spam_message(msg):
    return is_spam(calc_message(msg))
def report_spam_message(msg):
    report(msg, calc_message(msg))

In [82]:
# register spam messages
for msg in spams:
    for i in range(4):
        report_spam_message(msg)



(False, 0.9036066533202494)

In [78]:
is_spam_message(good[2])

In [83]:
is_spam_message(spams[2])

(True, 1.0000000000000004)

In [95]:
is_spam_message('Want to eat some cake? meet me at the coff. see you there!')

(False, 0.9627453035194151)